In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
import pickle
import os
import json # JSON 출력을 위해 추가
from sklearn.preprocessing import StandardScaler # 스케일러를 위해 추가

# matplotlib 한글 설정 (필요시 이 파일에도 포함)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# --- Custom JSON Encoder for NumPy types ---
class NumpyEncoder(json.JSONEncoder):
    """
    Custom JSON encoder that handles NumPy data types.
    Converts np.integer to int, np.floating to float, and np.ndarray to list.
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        # Handle bool_ separately as np.bool_ is also a subclass of np.integer
        elif isinstance(obj, np.bool_):
            return bool(obj)
        return json.JSONEncoder.default(self, obj)


# --- Precalculated Cluster Means (Identical to your original code) ---
# ... (HE_HP_precalculated_means, HE_DM_precalculated_means, obe_precalculated_means) ...
# 그대로 유지

HE_HP_precalculated_means = {
    1: pd.DataFrame({
        '1주일 간 음주 빈도': [0.167185, 0.239176, 3.172158, 1.363797, 0.304851, 0.318776, 0.126048],
        '하루 평균 흡연량': [0.321646, 0.312566, 2.000000, 20.489297, 0.751894, 0.548831, 0.317757],
        '1주일 간 걷기 일수': [0.863567, 6.208025, 4.344316, 3.593272, 2.034091, 6.138927, 0.781931],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [2.000000, 3.000000, 2.163088, 2.080349, 0.895143, 1.871035, 3.000000]
    }, index=[0, 1, 2, 3, 4, 5, 6]),
    2: pd.DataFrame({
        '1주일 간 음주 빈도': [0.284722, 2.500000, 0.858974, 0.150502, 0.221657, 5.500000, 0.165176],
        '하루 평균 흡연량': [0.633333, 1.149606, 19.418803, 0.337793, 0.470769, 4.475000, 0.191214],
        '1주일 간 걷기 일수': [6.180556, 4.299213, 3.709402, 1.187291, 1.098462, 4.075000, 6.297158],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [1.796187, 2.448819, 2.183140, 3.000000, 1.785248, 2.387500, 3.000000]
    }, index=[0, 1, 2, 3, 4, 5, 6]),
    3: pd.DataFrame({
        '1주일 간 음주 빈도': [0.763355, 0.468248, 1.676724, 0.647671, 5.500000, 0.515860, 0.553569],
        '하루 평균 흡연량': [1.145833, 0.461916, 19.193966, 0.774869, 5.348837, 0.309278, 0.989384],
        '1주일 간 걷기 일수': [5.069444, 1.302211, 3.288793, 6.057592, 4.203488, 6.268041, 1.171975],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [0.925590, 3.000000, 2.207806, 2.000000, 2.347993, 3.000000, 1.914155]
    }, index=[0, 1, 2, 3, 4, 5, 6]),
    4: pd.DataFrame({
        '1주일 간 음주 빈도': [2.756947, 0.653964, 5.500000, 0.479380, 0.589461, 0.454250],
        '하루 평균 흡연량': [20.293194, 0.917178, 2.215827, 0.298479, 0.627753, 0.253644],
        '1주일 간 걷기 일수': [3.701571, 1.303681, 3.856115, 6.292776, 6.162996, 1.192420],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [2.301771, 1.801597, 2.403903, 3.000000, 1.854179, 3.000000]
    }, index=[0, 1, 2, 3, 4, 5])
}

HE_DM_precalculated_means = {
    1: pd.DataFrame({
        '1주일 간 음주 빈도': [0.150946, 0.372085, 1.158756, 0.228997, 5.500000, 2.500000, 0.237966],
        '하루 평균 흡연량': [0.323770, 0.541776, 19.725768, 0.288396, 5.622449, 1.360502, 0.423890],
        '1주일 간 걷기 일수': [0.927049, 5.964267, 2.933806, 6.161263, 3.537415, 3.819749, 0.981501],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [3.000000, 1.771406, 2.114544, 3.000000, 2.211099, 2.249319, 1.774280]
    }, index=[0, 1, 2, 3, 4, 5, 6]),
    2: pd.DataFrame({
        '1주일 간 음주 빈도': [0.512626, 0.579406, 5.500000, 1.479076, 0.455288, 0.619097],
        '하루 평균 흡연량': [0.431724, 0.700143, 5.978166, 18.295597, 0.309375, 0.690799],
        '1주일 간 걷기 일수': [6.137931, 0.982783, 3.187773, 2.732704, 1.046875, 6.037707],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [3.000000, 1.842255, 2.362445, 2.143043, 3.000000, 1.812732]
    }, index=[0, 1, 2, 3, 4, 5]),
    3: pd.DataFrame({
        '1주일 간 음주 빈도': [0.145881, 0.142308, 0.546853, 1.466873, 5.500000, 0.098474],
        '하루 평균 흡연량': [0.161702, 0.342857, 0.309091, 20.496350, 6.471264, 0.170241],
        '1주일 간 걷기 일수': [0.331915, 1.023810, 5.949495, 2.905109, 2.977011, 0.239946],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [3.000000, 0.886941, 2.630303, 2.296130, 2.390805, 2.000000]
    }, index=[0, 1, 2, 3, 4, 5])
}

obe_precalculated_means = {
    1: pd.DataFrame({
        '1주일 간 음주 빈도': [0.041231, 0.499675, 0.140793, 4.500000, 0.955301, 0.043403],
        '하루 평균 흡연량': [0.000000, 0.253165, 0.078788, 7.933333, 17.162162, 0.046875],
        '1주일 간 걷기 일수': [0.144033, 5.886076, 0.454545, 2.766667, 3.513514, 0.138889],
        '하루 평균 채소 섭취 빈도': [3.000000, 2.239894, 0.892453, 2.080913, 2.135061, 2.000000]
    }, index=[0, 1, 2, 3, 4, 5]),
    2: pd.DataFrame({
        '1주일 간 음주 빈도': [1.437870, 0.338664, 0.565225, 0.397819, 5.500000],
        '하루 평균 흡연량': [16.953177, 0.164277, 0.426172, 0.364297, 7.000000],
        '1주일 간 걷기 일수': [2.856187, 3.597969, 6.012378, 0.932149, 3.066327],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [2.208769, 3.000000, 1.795531, 1.804506, 2.276950]
    }, index=[0, 1, 2, 3, 4]),
    3: pd.DataFrame({
        '1주일 간 음주 빈도': [0.378722, 0.526020, 0.595941, 5.500000, 1.447802, 0.462214],
        '하루 평균 흡연량': [0.326613, 0.415435, 0.581359, 4.986486, 19.017857, 0.407733],
        '1주일 간 걷기 일수': [0.987903, 0.957307, 5.990521, 3.567568, 2.852679, 6.130053],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [3.000000, 1.779103, 1.809968, 2.371122, 2.212353, 3.000000]
    }, index=[0, 1, 2, 3, 4, 5]),
    4: pd.DataFrame({
        '1주일 간 음주 빈도': [0.519089, 0.550410, 2.104251, 0.601160, 0.417795, 5.500000, 1.564362],
        '하루 평균 흡연량': [0.609069, 0.366864, 34.342105, 0.599757, 0.407108, 3.816667, 16.915825],
        '1주일 간 걷기 일수': [1.055147, 6.143491, 3.000000, 5.928224, 1.080775, 3.638889, 2.787879],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [1.812550, 3.000000, 2.289474, 1.808696, 3.000000, 2.355556, 2.102071]
    }, index=[0, 1, 2, 3, 4, 5, 6]),
    5: pd.DataFrame({
        '1주일 간 음주 빈도': [0.172308, 0.832751, 0.241978, 5.500000, 2.500000, 0.380881, 0.341026],
        '하루 평균 흡연량': [0.200000, 21.196970, 0.914286, 4.147059, 1.562500, 0.330097, 1.216667],
        '1주일 간 걷기 일수': [5.973333, 3.090909, 5.062857, 3.323529, 4.734375, 0.961165, 0.550000],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [3.000000, 1.890619, 1.764055, 2.189041, 2.069178, 3.000000, 1.813014]
    }, index=[0, 1, 2, 3, 4, 5, 6]),
    6: pd.DataFrame({
        '1주일 간 음주 빈도': [0.175570, 0.255583, 2.500000, 0.225175, 5.500000, 0.913462],
        '하루 평균 흡연량': [0.064815, 0.655914, 5.400000, 0.345455, 9.466667, 22.350000],
        '1주일 간 걷기 일수': [0.435185, 6.139785, 5.466667, 0.563636, 3.133333, 2.400000],
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': [3.000000, 2.174311, 2.333333, 1.796090, 2.228493, 2.042740]
    }, index=[0, 1, 2, 3, 4, 5])
}

# --- Custom Cluster Descriptions ---
CUSTOM_CLUSTER_DESCRIPTIONS = {
    'hypertension': {
        1: { # HE_HP2 = 1 (정상 혈압)
            0: "음주, 흡연, 걷기 모두 낮은 수준이지만 채소 섭취는 중간 정도입니다. 전반적으로 건강 습관이 좋지 않은 편으로, 특히 신체 활동이 매우 부족합니다. 이 군집은 건강 관리에 주의가 필요한 그룹으로 보입니다.",
            1: "음주와 흡연이 매우 낮고, 걷기는 매우 활발하며 채소 섭취도 높은 건강 지향적인 그룹입니다. 생활 습관 측면에서 가장 이상적인 군집 중 하나로 평가됩니다. 전반적으로 매우 건강한 생활 방식을 유지하고 있습니다.",
            2: "음주 횟수가 많고, 흡연량은 낮으며, 걷기는 중간 수준, 채소 섭취는 중간 정도입니다. 음주 습관을 개선할 필요가 있는 그룹입니다. 이 군집은 음주 빈도 조절을 통해 건강을 개선할 수 있습니다.",
            3: "음주와 걷기는 중간 수준이지만, 흡연량이 매우 높습니다. 채소 섭취는 중간 정도입니다. 흡연 습관 개선이 시급한 고위험군으로, 흡연으로 인한 건강 문제가 우려됩니다.",
            4: "음주와 흡연이 낮은 수준이고, 걷기는 낮은 편이며 채소 섭취도 낮습니다. 전반적으로 건강 습관이 부족한 그룹입니다. 이 군집은 신체 활동과 식습관 개선이 필요한 그룹입니다.",
            5: "음주와 흡연이 낮은 수준이고, 걷기는 매우 활발하며 채소 섭취는 중간 정도입니다. 흡연과 음주는 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 전반적으로 균형 잡힌 건강 습관을 가지고 있습니다.",
            6: "음주, 흡연, 걷기 모두 매우 낮은 수준이지만 채소 섭취는 높은 편입니다. 전반적으로 활동량이 매우 부족한 그룹으로, 특히 신체 활동 증진이 필요합니다. 건강한 식습관을 유지하고 있지만, 운동 부족이 문제입니다.",
        },
        2: { # HE_HP2 = 2 (주의 혈압)
            0: "음주와 흡연은 낮은 수준이지만, 걷기 활동은 매우 활발하며 채소 섭취는 낮은 편입니다. 운동은 잘 하고 있지만, 식습관 개선이 필요한 그룹입니다. 전반적인 건강 습관은 양호하나 채소 섭취 증진이 중요합니다.",
            1: "음주와 흡연은 중간 수준이고, 걷기 및 채소 섭취도 중간 정도입니다. 전반적으로 중간 수준의 건강 습관을 가진 그룹입니다. 특정 문제가 두드러지지 않지만, 전반적인 개선 노력이 필요합니다.",
            2: "음주는 낮지만 흡연량이 매우 높고, 걷기와 채소 섭취는 중간 수준입니다. 흡연 습관 개선이 시급한 고위험군으로, 혈압 관리에 있어 특히 주의가 필요합니다. 흡연으로 인한 건강 악화 가능성이 높습니다.",
            3: "음주와 흡연은 낮은 수준이고, 걷기는 낮은 편이며 채소 섭취는 높은 편입니다. 활동량이 부족하지만, 식습관은 비교적 건강한 그룹입니다. 신체 활동량 증진에 집중해야 합니다.",
            4: "음주와 흡연이 낮은 수준이고, 걷기는 낮은 편이며 채소 섭취도 낮은 편입니다. 전반적으로 건강 습관이 부족한 그룹입니다. 특히 운동과 식습관 전반의 개선이 필요합니다.",
            5: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기 및 채소 섭취는 중간 정도입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            6: "음주와 흡연은 낮은 수준이지만, 걷기 활동이 매우 활발하며 채소 섭취도 높은 편입니다. 매우 건강한 생활 습관을 가진 그룹입니다. 전반적으로 건강 관리를 잘 하고 있는 모범적인 그룹입니다.",
        },
        3: { # HE_HP2 = 3 (고혈압 전단계)
            0: "음주와 흡연은 낮고, 걷기는 높지만 채소 섭취는 낮은 편입니다. 걷기 운동은 잘 하지만, 식습관 개선이 필요한 그룹입니다. 건강한 신체 활동과 식습관의 균형이 중요합니다.",
            1: "음주와 흡연은 낮은 수준이고, 걷기는 낮지만 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요한 그룹입니다.",
            2: "음주는 중간 수준, 흡연량은 매우 높고, 걷기와 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군이며, 음주 습관 관리도 필요합니다. 전반적인 생활 습관 개선이 절실합니다.",
            3: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 중간 정도입니다. 음주와 흡연은 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 전반적으로 균형 잡힌 건강 습관을 가지고 있습니다.",
            4: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기 및 채소 섭취는 중간 정도입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            5: "음주와 흡연은 낮은 수준이지만, 걷기 활동은 매우 활발하며 채소 섭취는 높은 편입니다. 매우 건강한 생활 습관을 가진 그룹입니다. 가장 건강한 생활 방식을 유지하고 있습니다.",
            6: "음주와 흡연은 낮고, 걷기는 낮은 편이며 채소 섭취도 낮습니다. 전반적으로 건강 습관이 부족한 그룹입니다. 특히 운동과 식습관 전반의 개선이 필요합니다.",
        },
        4: { # HE_HP2 = 4 (고혈압)
            0: "음주와 걷기는 중간 수준이지만, 흡연량이 매우 높고 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 초고위험군입니다. 혈압 관리에 있어 가장 집중적인 개입이 필요한 그룹입니다.",
            1: "음주, 흡연, 걷기, 채소 섭취 모두 낮은 수준입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
            2: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간 정도입니다. 음주와 흡연 습관 개선이 필요한 고위험군입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            3: "음주와 흡연은 낮은 수준이지만, 걷기 활동이 매우 활발하며 채소 섭취는 높은 편입니다. 매우 건강한 생활 습관을 가진 그룹입니다. 고혈압 상태임에도 건강 관리를 잘 하고 있습니다.",
            4: "음주와 흡연은 낮고, 걷기는 높지만 채소 섭취는 낮은 편입니다. 걷기 운동은 잘 하지만, 식습관 개선이 필요한 그룹입니다. 건강한 신체 활동과 식습관의 균형이 중요합니다.",
            5: "음주와 흡연은 낮은 수준이고, 걷기는 낮지만 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요한 그룹입니다.",
        }
    },
    'diabetes': {
        1: { # HE_DM_HbA1c2 = 1 (정상)
            0: "음주, 흡연, 걷기 모두 낮은 수준이지만 채소 섭취는 높습니다. 전반적으로 활동량이 부족하지만, 식습관은 건강한 그룹입니다. 운동량 증진이 필요합니다.",
            1: "음주와 흡연은 낮고, 걷기는 중간이며 채소 섭취는 낮은 편입니다. 신체 활동은 비교적 꾸준하지만, 식습관 개선이 필요합니다. 전반적인 건강 관리에 주의가 필요합니다.",
            2: "음주는 중간, 흡연량은 매우 높고, 걷기는 낮으며 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
            3: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 높습니다. 음주와 흡연은 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 가장 이상적인 건강 습관을 가진 군집입니다.",
            4: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            5: "음주와 걷기는 중간 수준이고, 흡연은 낮으며 채소 섭취는 중간입니다. 전반적으로 균형 잡힌 건강 습관을 가지고 있습니다. 특별히 문제되는 습관 없이 적정 수준을 유지하고 있습니다.",
            6: "음주, 흡연, 걷기, 채소 섭취 모두 낮은 수준입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
        },
        2: { # HE_DM_HbA1c2 = 2 (당뇨 전단계)
            0: "음주와 흡연은 낮은 수준이지만, 걷기 활동은 매우 활발하며 채소 섭취는 높습니다. 운동은 잘 하고 있지만, 전반적인 건강 습관은 양호합니다. 건강 관리를 잘 하고 있는 그룹입니다.",
            1: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 부족한 그룹입니다. 특히 운동과 식습관 전반의 개선이 필요합니다.",
            2: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            3: "음주는 중간, 흡연량은 매우 높고, 걷기는 낮으며 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
            4: "음주와 흡연은 낮은 수준이고, 걷기는 낮지만 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요한 그룹입니다.",
            5: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 낮은 편입니다. 걷기 운동은 잘 하지만, 식습관 개선이 필요한 그룹입니다. 건강한 신체 활동과 식습관의 균형이 중요합니다.",
        },
        3: { # HE_DM_HbA1c2 = 3 (당뇨병)
            0: "음주, 흡연, 걷기 모두 낮은 수준이지만 채소 섭취는 높습니다. 전반적으로 활동량이 부족하지만, 식습관은 건강한 그룹입니다. 운동량 증진이 필요합니다.",
            1: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
            2: "음주와 흡연은 낮지만, 걷기는 중간이며 채소 섭취는 중간 정도입니다. 흡연과 음주는 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 당뇨병 관리에 있어 좋은 습관을 가지고 있습니다.",
            3: "음주는 중간, 흡연량은 매우 높고, 걷기는 낮으며 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
            4: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            5: "음주, 흡연, 걷기 모두 낮은 수준이지만 채소 섭취는 중간 정도입니다. 전반적으로 활동량이 매우 부족한 그룹으로, 특히 신체 활동 증진이 필요합니다. 건강한 식습관을 유지하고 있지만, 운동 부족이 문제입니다.",
        }
    },
    'obesity': {
        1: { # HE_obe2 = 1 (저체중)
            0: "음주, 흡연, 걷기 모두 매우 낮은 수준이지만 채소 섭취는 높습니다. 활동량 부족이 심각하지만, 채소 위주의 식습관을 유지하고 있습니다. 건강 증진을 위해 신체 활동을 늘려야 합니다.",
            1: "음주와 흡연은 낮고, 걷기는 중간이며 채소 섭취는 중간 정도입니다. 전반적으로 건강 습관이 비교적 양호한 그룹입니다. 특별히 문제되는 습관 없이 적정 수준을 유지하고 있습니다.",
            2: "음주, 흡연, 걷기 모두 낮은 수준이고 채소 섭취도 낮습니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 특히 운동과 식습관 전반의 개선이 절실합니다.",
            3: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주 습관과 흡연 습관 개선이 필요한 고위험군입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            4: "음주는 낮지만 흡연량이 매우 높고, 걷기와 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
            5: "음주, 흡연, 걷기 모두 낮은 수준이지만 채소 섭취는 중간 정도입니다. 전반적으로 활동량이 매우 부족한 그룹으로, 특히 신체 활동 증진이 필요합니다. 건강한 식습관을 유지하고 있지만, 운동 부족이 문제입니다.",
        },
        2: { # HE_obe2 = 2 (정상 체중)
            0: "음주는 낮고, 걷기는 중간이며 채소 섭취는 높습니다. 흡연량은 매우 낮습니다. 신체 활동과 식습관 모두 양호하여 건강 관리를 잘 하고 있는 그룹입니다.",
            1: "음주, 흡연, 걷기, 채소 섭취 모두 낮은 수준입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
            2: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 낮은 편입니다. 걷기 운동은 잘 하지만, 식습관 개선이 필요한 그룹입니다. 건강한 신체 활동과 식습관의 균형이 중요합니다.",
            3: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 특히 운동과 식습관 전반의 개선이 필요합니다.",
            4: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
        },
        3: { # HE_obe2 = 3 (과체중)
            0: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요한 그룹입니다.",
            1: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
            2: "음주와 흡연은 낮지만, 걷기는 중간이며 채소 섭취는 낮은 편입니다. 걷기 운동은 어느 정도 하지만, 전반적인 식습관 개선이 필요합니다. 건강한 생활 습관을 위해 노력해야 합니다.",
            3: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            4: "음주는 중간, 흡연량이 매우 높고, 걷기는 낮으며 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
            5: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 높습니다. 음주와 흡연은 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 가장 이상적인 건강 습관을 가진 군집입니다.",
        },
        4: { # HE_obe2 = 4 (비만)
            0: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
            1: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 높습니다. 음주와 흡연은 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 비만 상태임에도 건강한 생활 습관을 유지하고 있습니다.",
            2: "음주는 중간이지만 흡연량이 매우 높고, 걷기와 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 초고위험군으로, 비만과 흡연 모두 관리해야 합니다. 건강 문제가 심각할 수 있습니다.",
            3: "음주와 흡연은 낮고, 걷기는 중간이며 채소 섭취는 낮은 편입니다. 걷기 운동은 어느 정도 하지만, 전반적인 식습관 개선이 필요합니다. 건강한 생활 습관을 위해 노력해야 합니다.",
            4: "음주와 흡연은 낮고, 걷기는 낮지만 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요한 그룹입니다.",
            5: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            6: "음주는 중간이지만 흡연량이 매우 높고, 걷기는 낮으며 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
        },
        5: { # HE_obe2 = 5 (고도 비만)
            0: "음주와 흡연은 낮고, 걷기는 중간이며 채소 섭취는 높습니다. 전반적으로 신체 활동과 식습관 모두 양호하여 건강 관리를 잘 하고 있는 그룹입니다. 고도 비만임에도 건강한 생활 습관을 유지하고 있습니다.",
            1: "음주는 낮지만 흡연량이 매우 높고, 걷기는 중간이며 채소 섭취는 낮은 편입니다. 흡연 습관 개선이 시급한 초고위험군으로, 고도 비만과 흡연 모두 관리해야 합니다. 건강 문제가 심각할 수 있습니다.",
            2: "음주와 흡연은 낮고, 걷기는 중간이며 채소 섭취는 낮은 편입니다. 신체 활동은 비교적 꾸준하지만, 식습관 개선이 필요합니다. 전반적인 건강 관리에 주의가 필요합니다.",
            3: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            4: "음주와 걷기는 중간 수준이고, 흡연은 중간이며 채소 섭취는 중간입니다. 전반적으로 균형 잡힌 건강 습관을 가지고 있습니다. 특별히 문제되는 습관 없이 적정 수준을 유지하고 있습니다.",
            5: "음주와 흡연은 낮고, 걷기는 낮지만 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요한 그룹입니다.",
            6: "음주와 걷기는 낮은 수준이고, 흡연량은 중간이며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 부족한 그룹입니다. 특히 운동과 식습관 전반의 개선이 절실합니다.",
        },
        6: { # HE_obe2 = 6 (초고도 비만)
            0: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 높습니다. 활동량이 부족하지만, 채소 위주의 건강한 식습관을 가진 그룹입니다. 운동량 증진이 필요합니다.",
            1: "음주와 흡연은 낮고, 걷기는 높으며 채소 섭취는 중간입니다. 음주와 흡연은 적지만, 걷기 운동을 꾸준히 하는 건강 관리형 그룹입니다. 전반적으로 균형 잡힌 건강 습관을 가지고 있습니다.",
            2: "음주와 걷기는 중간 수준이고, 흡연은 중간이며 채소 섭취는 중간입니다. 전반적으로 균형 잡힌 건강 습관을 가지고 있습니다. 특별히 문제되는 습관 없이 적정 수준을 유지하고 있습니다.",
            3: "음주와 흡연은 낮고, 걷기는 낮으며 채소 섭취는 낮은 편입니다. 전반적으로 건강 습관이 매우 부족한 그룹입니다. 생활 습관 전반에 걸쳐 개선이 절실합니다.",
            4: "음주 횟수가 많고, 흡연량은 중간 수준이며 걷기와 채소 섭취는 중간입니다. 음주와 흡연 습관 개선이 필요한 고위험군으로 보입니다. 건강에 해로운 습관들을 개선해야 합니다.",
            5: "음주는 낮지만 흡연량이 매우 높고, 걷기는 낮으며 채소 섭취는 중간입니다. 흡연 습관 개선이 시급한 고위험군으로, 생활 습관 전반의 개선이 절실합니다. 특히 흡연으로 인한 건강 문제가 우려됩니다.",
        }
    }
}


# --- Functions for disease determination, cluster determination, and plotting ---
# (These functions remain largely the same, with plot_radar_chart returning filepath)

def determine_disease_groups(sbp, dbp, fglu, bmi):
    """
    주요 건강 지표를 기반으로 질병 그룹을 결정합니다.
    (원래 코드와 동일)
    """
    # 고혈압
    if sbp >= 140 or dbp >= 90:
        he_hp2 = 4
    elif 130 <= sbp <= 139 or 80 <= dbp <= 89:
        he_hp2 = 3
    elif 120 <= sbp <= 129 and dbp < 80:
        he_hp2 = 2
    else:
        he_hp2 = 1

    # 당뇨
    if fglu >= 126:
        he_dm = 3
    elif 100 <= fglu < 126:
        he_dm = 2
    else:
        he_dm = 1

    # 비만
    if bmi <= 18.5:
        he_bmi = 1
    elif 18.5 < bmi <= 22.9:
        he_bmi = 2
    elif 22.9 < bmi <= 24.9:
        he_bmi = 3
    elif 24.9 < bmi <= 29.9:
        he_bmi = 4
    elif 29.9 < bmi <= 34.9:
        he_bmi = 5
    else:
        he_bmi = 6

    return he_hp2, he_dm, he_bmi

def determine_cluster_with_precalculated_means(user_data_weekly_daily, group, disease, scaler, precalculated_means_dict, selected_cols):
    """
    사용자 데이터를 가장 가까운 사전 계산된 군집에 매핑합니다.
    (원래 코드와 동일)
    """
    group_int = int(group)
    if group_int not in precalculated_means_dict:
        return None, None

    cluster_means_precalculated_text_names = precalculated_means_dict[group_int].copy()
    text_to_internal_col_mapping = {
        '1주일 간 음주 빈도': 'BD1_11',
        '하루 평균 흡연량': 'tobacco',
        '1주일 간 걷기 일수': 'BE3_31',
        '하루 평균 채소류/버섯류/해조류 섭취 빈도': 'LS_VEG1'
    }

    cluster_means_precalculated_internal_names = pd.DataFrame(index=cluster_means_precalculated_text_names.index)
    for internal_col in selected_cols:
        text_col = next((k for k, v in text_to_internal_col_mapping.items() if v == internal_col), None)
        if text_col and text_col in cluster_means_precalculated_text_names.columns:
            cluster_means_precalculated_internal_names[internal_col] = cluster_means_precalculated_text_names[text_col]
        else:
            return None, None

    if cluster_means_precalculated_internal_names.isnull().values.any():
        return None, None

    # Apply the same transformations as done during scaler training for centroids
    # BD1_11 (weekly_alcohol) was transformed to annual
    # LS_VEG1 (daily_veg) was transformed to annual
    cluster_means_transformed = cluster_means_precalculated_internal_names.copy()
    cluster_means_transformed['BD1_11'] = cluster_means_transformed['BD1_11'] * 52
    cluster_means_transformed['LS_VEG1'] = cluster_means_transformed['LS_VEG1'] * 365

    scaled_precalculated_centroids = scaler.transform(cluster_means_transformed[selected_cols])

    # Scaling for user data should match the scaling used during training
    user_data_transformed = [
        user_data_weekly_daily[0] * 52, # Convert weekly alcohol to annual for BD1_11
        user_data_weekly_daily[1], # Daily smoking remains daily for tobacco
        user_data_weekly_daily[2],  # Weekly exercise remains weekly for BE3_31
        user_data_weekly_daily[3] * 365 # Convert daily veg to annual for LS_VEG1
    ]

    user_data_scaled = scaler.transform([user_data_transformed])
    distances = np.linalg.norm(scaled_precalculated_centroids - user_data_scaled, axis=1)
    closest_cluster = np.argmin(distances)
    cluster_means = precalculated_means_dict[group_int].iloc[closest_cluster]

    return closest_cluster, cluster_means

def plot_radar_chart(user_data, cluster_mean, categories, title, name, output_dir="aaaaaaaaa"):
    """
    레이더 차트를 그리고 이미지 파일 경로를 반환합니다.
    """
    os.makedirs(output_dir, exist_ok=True) # 출력 디렉토리 생성

    user_data_list = user_data if isinstance(user_data, list) else user_data.tolist()
    cluster_mean_list = cluster_mean if isinstance(cluster_mean, list) else cluster_mean.tolist()

    values_user = user_data_list[:]
    values_cluster = cluster_mean_list[:]

    # Ensure all values are floats for plotting
    values_user = [float(v) for v in values_user]
    values_cluster = [float(v) for v in values_cluster]

    all_values = values_user + values_cluster
    max_val = max(all_values) if all_values else 1 # Empty list fallback

    values_user = values_user + values_user[:1]
    values_cluster = values_cluster + values_cluster[:1]
    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.set_ylim(0, max_val * 1.2)
    ax.fill(angles, values_user, color='red', alpha=0.25, label=name)
    ax.plot(angles, values_user, color='red', linewidth=2)
    ax.fill(angles, values_cluster, color='blue', alpha=0.25, label='유형 평균')
    ax.plot(angles, values_cluster, color='blue', linewidth=2)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=10)
    ax.set_title(title, size=15, color='black', y=1.1)
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
    ax.grid(True)

    filename = f"{title.replace(' ', '_').replace('/', '_').replace(':', '_')}.png" # ':' 제거 추가
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath)
    plt.close(fig) # 메모리 해제
    return filepath # 저장된 파일 경로 반환

# --- Scaler Training and Saving Function ---
def train_and_save_scalers(scaler_filepath="scalers.pkl"):
    """
    더미 데이터를 사용하여 각 질병 그룹에 대한 StandardScaler를 학습하고 저장합니다.
    실제 사용 시에는 실제 훈련 데이터를 사용해야 합니다.
    """
    print(f"'{scaler_filepath}'를 생성하고 있습니다. (더미 데이터 사용)")

    scalers = {}
    selected_cols = ['BD1_11', 'tobacco', 'BE3_31', 'LS_VEG1']

    # Dummy data generation for scaler training
    # This data should ideally resemble the distribution of your actual training data
    # that was used to create the clusters.
    dummy_data = pd.DataFrame({
        'BD1_11': np.random.uniform(0, 7, 1000),  # Weekly alcohol (0-7 days)
        'tobacco': np.random.uniform(0, 30, 1000), # Daily smoking (0-30 cigarettes)
        'BE3_31': np.random.uniform(0, 7, 1000),   # Weekly exercise (0-7 days)
        'LS_VEG1': np.random.uniform(1, 3, 1000)   # Daily veg (1-3 times)
    })

    # Apply the same transformations that are applied to user data and cluster means
    # BD1_11 (weekly_alcohol) is transformed to annual in analyze_user_lifestyle
    # LS_VEG1 (daily_veg) is transformed to annual in analyze_user_lifestyle
    # So, scalers should be fit on transformed data
    transformed_dummy_data = dummy_data.copy()
    transformed_dummy_data['BD1_11'] = transformed_dummy_data['BD1_11'] * 52
    transformed_dummy_data['LS_VEG1'] = transformed_dummy_data['LS_VEG1'] * 365

    # Iterate through all disease groups and fit a scaler for each
    # This assumes that a single scaler is used per disease type regardless of subgroup
    # If different scalers are needed per subgroup (e.g., hypertension group 1, hypertension group 2),
    # the logic here would need to be more complex to create and fit individual scalers.
    # For now, we'll create a scaler for each (disease, group_int) pair
    disease_groups_for_scaler = [
        ('hypertension', 1), ('hypertension', 2), ('hypertension', 3), ('hypertension', 4),
        ('diabetes', 1), ('diabetes', 2), ('diabetes', 3),
        ('obesity', 1), ('obesity', 2), ('obesity', 3), ('obesity', 4), ('obesity', 5), ('obesity', 6)
    ]

    for disease, group_int in disease_groups_for_scaler:
        scaler = StandardScaler()
        scaler.fit(transformed_dummy_data[selected_cols]) # Fit on the transformed dummy data
        scalers[(disease, group_int)] = scaler

    with open(scaler_filepath, 'wb') as f:
        pickle.dump(scalers, f)
    print(f"'{scaler_filepath}' 생성이 완료되었습니다.")


def analyze_user_lifestyle(user_data_input, scaler_filepath="scalers.pkl"):
    """
    사용자 데이터를 입력받아 생활습관 군집 분석을 수행하고 결과를 JSON 형태로 반환합니다.

    Args:
        user_data_input (dict): 사용자 입력 데이터 (예: {'name': '홍길동', 'sbp': 120, ...})
        scaler_filepath (str): 스케일러 파일 경로

    Returns:
        dict: 분석 결과가 담긴 딕셔너리 (JSON으로 직렬화 가능)
    """
    results = {}

    selected_cols = ['BD1_11', 'tobacco', 'BE3_31', 'LS_VEG1']
    item_names = ['음주', '흡연', '걷기', '채소']

    scalers = {}

    # 스케일러 로드 시도
    if os.path.exists(scaler_filepath):
        try:
            with open(scaler_filepath, 'rb') as f:
                scalers = pickle.load(f)
        except Exception as e:
            results['error'] = f"스케일러 로드 중 오류 발생: {e}. 'train_and_save_scalers.py'를 다시 실행해야 할 수 있습니다."
            return results
    else:
        results['error'] = f"스케일러 파일 '{scaler_filepath}'이(가) 존재하지 않습니다. 먼저 이 스크립트를 실행하여 스케일러를 생성해주세요."
        return results

    # 사용자 입력 데이터 추출
    try:
        name = user_data_input['user_name']
        sbp = user_data_input['HE_sbp1']
        dbp = user_data_input['HE_dbp1']
        fglu = user_data_input['HE_glu']
        bmi = user_data_input['HE_BMI']
        weekly_alcohol = user_data_input['weekly_alcohol']
        daily_smoking = user_data_input['daily_smoking']
        weekly_exercise = user_data_input['weekly_exercise']
        daily_veg = user_data_input['daily_veg']
    except KeyError as e:
        results['error'] = f"필수 사용자 입력 데이터가 누락되었습니다: {e}. 모든 항목을 입력해주세요."
        return results
    except Exception as e:
        results['error'] = f"사용자 입력 데이터 처리 중 오류 발생: {e}"
        return results

    he_hp2, he_dm, he_bmi = determine_disease_groups(sbp, dbp, fglu, bmi)

    disease_labels = {
        'hypertension': {1: "정상 혈압", 2: "주의 혈압", 3: "고혈압 전단계", 4: "고혈압"},
        'diabetes': {1: "정상", 2: "당뇨 전단계", 3: "당뇨"},
        'obesity': {1: "저체중", 2: "정상 체중", 3: "과체중", 4: "비만", 5: "고도비만", 6: "초고도비만"}
    }

    diseases = [
        ('hypertension', he_hp2, HE_HP_precalculated_means, 'HE_HP2'),
        ('diabetes', he_dm, HE_DM_precalculated_means, 'HE_DM'),
        ('obesity', he_bmi, obe_precalculated_means, 'HE_BMI_class')
    ]

    user_data_lifestyle = [weekly_alcohol, daily_smoking, weekly_exercise, daily_veg]

    analysis_results = []

    for disease, group, precalculated_means, col_name in diseases:
        group_int = int(group)
        disease_status = disease_labels[disease][group_int]

        current_disease_result = {
            "disease_name": disease,
            "status_label": disease_status,
            "status_code": group_int,
            "cluster_info": None,
            "radar_chart_image_path": None,
            "comparison_table": [],
            "analysis_comments": []
        }

        # 해당 그룹의 스케일러 로드
        scaler_key = (disease, group_int)
        if scaler_key not in scalers:
            current_disease_result['error'] = f"오류: {disease} 그룹 {group_int}에 대한 스케일러가 로드되지 않았습니다."
            analysis_results.append(current_disease_result)
            continue

        current_scaler = scalers[scaler_key]

        cluster, cluster_means = determine_cluster_with_precalculated_means(
            user_data_lifestyle, group_int, disease, current_scaler, precalculated_means, selected_cols
        )

        if cluster is not None:
            display_cluster = cluster + 1 # 0-indexed cluster to 1-indexed for display

            # 군집 특징 추출 및 출력 (Using custom descriptions)
            cluster_description = CUSTOM_CLUSTER_DESCRIPTIONS.get(disease, {}).get(group_int, {}).get(cluster, "이 군집에 대한 사용자 정의 설명이 없습니다.")

            current_disease_result["cluster_info"] = {
                "cluster_number": display_cluster,
                "description": cluster_description
            }

            # 레이더 차트 생성 및 경로 저장
            chart_title = f"{name}님의 생활습관 vs {disease_status} 그룹 {display_cluster} 평균"
            image_path = plot_radar_chart(
                user_data_lifestyle,
                cluster_means.tolist(), # Ensure it's a list for plotting
                item_names,
                chart_title,
                name
            )
            current_disease_result["radar_chart_image_path"] = image_path

            # 항목 이름 매핑: 보여줄 이름 :양방향_화살표: 실제 column 이름
            item_display_map = {
                '음주': '1주일 간 음주 빈도',
                '흡연': '하루 평균 흡연량',
                '걷기': '1주일 간 걷기 일수',
                '채소': '하루 평균 채소류/버섯류/해조류 섭취 빈도'
            }
            # 사용자의 수치와 군집의 수치가 적힌 표 출력
            table_data = []
            # for i, item_name in enumerate(item_names):
            #     table_data.append({
            #         '항목': item_name,
            #         f'{name}님 수치': round(float(user_data_lifestyle[i]), 2), # Ensure float for rounding
            #         f'군집 {display_cluster} 평균 수치': round(float(cluster_means[item_name]), 2) # Ensure float for rounding
            #     })
                
            for display_name in item_names:
                original_col = item_display_map[display_name]
                table_data.append({
                    '항목': display_name,
                    f'{name}님 수치': round(float(user_data_lifestyle[item_names.index(display_name)]), 2),
                    f'군집 평균 수치': round(float(cluster_means[original_col]), 2)
                })    
                
                
            current_disease_result["comparison_table"] = table_data

            # 사용자 수치와 군집 수치를 비교하는 문구 출력 (개인화된 분석)
            comparison_comments = []
            # Note: For comparisons, use the original scale, not the transformed one used for scaling.
            # user_data_lifestyle is already in weekly/daily format.
            # cluster_means is also from the precalculated_means_dict, so it's in weekly/daily/raw units.

            if user_data_lifestyle[0] > cluster_means['1주일 간 음주 빈도']:
                comparison_comments.append(f"군집 평균({cluster_means['1주일 간 음주 빈도']:.2f}회)에 비해 음주 빈도({user_data_lifestyle[0]:.2f}회)가 많으므로 주의가 필요합니다.")
            elif user_data_lifestyle[0] < cluster_means['1주일 간 음주 빈도']:
                comparison_comments.append(f"군집 평균({cluster_means['1주일 간 음주 빈도']:.2f}회)보다 음주 빈도({user_data_lifestyle[0]:.2f}회)가 적습니다. 좋은 습관입니다.")
            else:
                comparison_comments.append(f"음주 빈도({user_data_lifestyle[0]:.2f}회)가 군집 평균과 유사합니다.")

            if user_data_lifestyle[1] > cluster_means['하루 평균 흡연량']:
                comparison_comments.append(f"군집 평균({cluster_means['하루 평균 흡연량']:.2f}개비)에 비해 흡연량({user_data_lifestyle[1]:.2f}개비)이 많으므로 주의가 필요합니다.")
            elif user_data_lifestyle[1] < cluster_means['하루 평균 흡연량']:
                comparison_comments.append(f"군집 평균({cluster_means['하루 평균 흡연량']:.2f}개비)보다 흡연량({user_data_lifestyle[1]:.2f}개비)이 적습니다. 좋은 습관입니다.")
            else:
                comparison_comments.append(f"흡연량({user_data_lifestyle[1]:.2f}개비)이 군집 평균과 유사합니다.")

            if user_data_lifestyle[2] < cluster_means['1주일 간 걷기 일수']:
                comparison_comments.append(f"군집 평균({cluster_means['1주일 간 걷기 일수']:.2f}일)에 비해 걷기 일수({user_data_lifestyle[2]:.2f}일)가 적으므로 활동량을 늘리는 것이 좋습니다.")
            elif user_data_lifestyle[2] > cluster_means['1주일 간 걷기 일수']:
                comparison_comments.append(f"군집 평균({cluster_means['1주일 간 걷기 일수']:.2f}일)보다 걷기 일수({user_data_lifestyle[2]:.2f}일)가 많습니다. 활동량이 충분합니다.")
            else:
                comparison_comments.append(f"걷기 일수({user_data_lifestyle[2]:.2f}일)가 군집 평균과 유사합니다.")

            if user_data_lifestyle[3] < cluster_means['하루 평균 채소류/버섯류/해조류 섭취 빈도']:
                comparison_comments.append(f"군집 평균({cluster_means['하루 평균 채소류/버섯류/해조류 섭취 빈도']:.2f}회/일)에 비해 채소류 섭취 빈도({user_data_lifestyle[3]:.2f}회/일)가 적으므로 섭취량을 늘리는 것을 권장합니다.")
            elif user_data_lifestyle[3] > cluster_means['하루 평균 채소류/버섯류/해조류 섭취 빈도']:
                comparison_comments.append(f"군집 평균({cluster_means['하루 평균 채소류/버섯류/해조류 섭취 빈도']:.2f}회/일)보다 채소류 섭취 빈도({user_data_lifestyle[3]:.2f}회/일)가 많습니다. 건강한 식습관입니다.")
            else:
                comparison_comments.append(f"채소류 섭취 빈도({user_data_lifestyle[3]:.2f}회/일)가 군집 평균과 유사합니다.")

            current_disease_result["analysis_comments"] = comparison_comments

        analysis_results.append(current_disease_result)

    results['analysis_results'] = analysis_results
    return results



   

In [2]:
# 실제 서비스 호출 예시
if __name__ == "__main__":
    scaler_filepath = "scalers.pkl"


    # 이 부분은 실제 서비스에서 사용자로부터 입력받는 데이터를 시뮬레이션합니다.
    # 웹/앱에서는 보통 JSON 형태의 요청 바디로 이런 데이터를 보냅니다.
    sample_user_data = {
        'user_name': '김미정',
        'HE_sbp1': 140,
        'HE_dbp1': 90,
        'HE_glu': 99,
        'HE_BMI': 28.5,
        'weekly_alcohol': 3.0,
        'daily_smoking': 4.0,
        'weekly_exercise': 5.0,
        'daily_veg': 2.0
    }

    # 분석 함수 호출
    analysis_output = analyze_user_lifestyle(sample_user_data, scaler_filepath=scaler_filepath)

    # 결과 출력 (서비스에서는 이 JSON을 HTTP 응답으로 보냄)
    print("--- 첫 번째 사용자 분석 결과 ---")
    print(json.dumps(analysis_output, indent=2, ensure_ascii=False, cls=NumpyEncoder)) # NumpyEncoder 사용

    # 에러가 있을 경우 에러 메시지 출력 (실제 서비스에서는 HTTP 500 에러 등으로 처리)
    if 'error' in analysis_output:
        print(f"\n오류 발생: {analysis_output['error']}")
    else:
        # 생성된 레이더 차트 이미지 파일 경로 출력 (실제 서비스에서는 이 경로를 통해 이미지를 띄움)
        print("\n--- 생성된 레이더 차트 이미지 파일 경로 ---")
        for res in analysis_output['analysis_results']:
            if res.get('radar_chart_image_path'):
                print(f"{res['disease_name']}: {res['radar_chart_image_path']}")
            else:
                print(f"{res['disease_name']}: 레이더 차트 생성 실패 또는 해당 없음")

--- 첫 번째 사용자 분석 결과 ---
{
  "analysis_results": [
    {
      "disease_name": "hypertension",
      "status_label": "고혈압",
      "status_code": 4,
      "cluster_info": {
        "cluster_number": 5,
        "description": "음주와 흡연은 낮고, 걷기는 높지만 채소 섭취는 낮은 편입니다. 걷기 운동은 잘 하지만, 식습관 개선이 필요한 그룹입니다. 건강한 신체 활동과 식습관의 균형이 중요합니다."
      },
      "radar_chart_image_path": "aaaaaaaaa\\김미정님의_생활습관_vs_고혈압_그룹_5_평균.png",
      "comparison_table": [
        {
          "항목": "음주",
          "김미정님 수치": 3.0,
          "군집 평균 수치": 0.59
        },
        {
          "항목": "흡연",
          "김미정님 수치": 4.0,
          "군집 평균 수치": 0.63
        },
        {
          "항목": "걷기",
          "김미정님 수치": 5.0,
          "군집 평균 수치": 6.16
        },
        {
          "항목": "채소",
          "김미정님 수치": 2.0,
          "군집 평균 수치": 1.85
        }
      ],
      "analysis_comments": [
        "군집 평균(0.59회)에 비해 음주 빈도(3.00회)가 많으므로 주의가 필요합니다.",
        "군집 평균(0.63개비)에 비해 흡연량(4.00개비)이 많으므로 주의가 필요합니다.",
        "군집 평균(6.16일)에 비해 걷기 일

c:\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
